## Константы и импорты

In [1]:
#!python -m pip uninstall numpy --yes
#!pip install "numpy<2.0"

In [2]:
import numpy as np
import time

from sklearn.ensemble import IsolationForest

import matplotlib.pyplot as plt
import warnings
import math

warnings.filterwarnings("ignore")

connect_treshold = 10
separate_treshold = 5
y_separate_treshold = 10
y_connect_treshold = 15

IMS = 0.297
Z_calibration = -5
Y_Calibration = 29

estimators = 100
max_features = 2

verticular_treshold = 3
verticular_len = 60

file_dir = "2024-Dec-04/2024-Dec-04/"

Trues = {"A" : {1 : [124, 124, 135], 2 : [124, 124, 135], 3 : [135, 124, 124]}, "B" : {1 : [114, 115, 266], 2 : [114, 115, 266], 3 : [266, 115, 114], 4 : [266, 115, 114]},
             "C" : {1 : [164, 178, 83], 2 : [164, 178, 83], 3 : [164, 83, 178], 4 : [164, 83, 178]}, "D" : {1 : [200, 130, 40], 2 : [200, 130, 40], 3 : [130, 40, 200], 4 : [200, 40, 130], 5 : [200, 40, 130]},
             "E" : {1 : [202, 197, 87], 2 : [197, 202, 87], 3 : [202, 87, 197], 4 : [202, 87, 197]}, "F" : {1 : [74, 79, 67], 2 : [74, 67, 79], 3 : [74, 67, 79]},
             "G" : {1 : [258, 123, 84], 2 : [258, 123, 84], 3 : [258, 84, 123], 4 : [258, 84, 123]}, "H" : {1 : [149, 147, 158], 2 : [149, 147, 158]},
             "I" : {1 : [64, 87, 113], 2 : [113, 87, 64], 3 : [113, 87, 64]}, "J" : {1 : [118, 95, 50], 2 : [118, 50, 95], 3 : [118, 50, 95]}}

Filenames = {"A" : {1 : [1, 2, 3], 2 : [1, 2, 3], 3 : [1, 2, 3]}, "B" : {1 : [1, 2, 3], 2 : [1, 2, 3], 3 : [1, 2, 3], 4 : [1, 2, 3]},
             "C" : {1 : [1, 2], 2 : [1, 2, 3], 3 : [1, 2, 3], 4 : [1, 2, 3]}, "D" : {1 : [1, 2, 3], 2 : [1, 2, 3], 3 : [1, 2, 3], 4 : [1, 2, 3], 5 : [1, 2, 3]},
             "E" : {1 : [1, 2, 3], 2 : [1, 2, 3], 3 : [1, 2, 3], 4 : [1, 2, 3]}, "F" : {1 : [1, 2, 3], 2 : [1, 2, 3], 3 : [1, 2, 3]},
             "G" : {1 : [1, 2, 3], 2 : [1, 2, 3], 3 : [1, 2, 3], 4 : [1, 2, 3]}, "H" : {1 : [1, 2, 3], 2 : [1, 2, 3]},
             "I" : {1 : [1, 2, 3], 2 : [1, 2, 3], 3 : [1, 2, 3]}, "J" : {1 : [1, 2, 3], 2 : [1, 2, 3], 3 : [1, 2, 3]}}

sensors_xyz = [[0, 0, 35], [0, 0, 90], [0, 0, 155],
               [0, 81, 290], [0, 145, 290], [0, 195, 290],
               [0, 293, 130], [0, 293, 90], [0, 293, 30]]

## Общие вспомогательные функции

In [3]:
def correct_filenames(Filenames):
    try:
        for letter in Filenames.keys():
            for state_num in Filenames[letter].keys():
                for logs_num in Filenames[letter][state_num]:
                    filename = file_dir + letter + "_" + str(state_num) + "_" + str(logs_num) + ".log"
                    file = open(filename)
                    file.close()
    except:
        return False     
    return True

def dist_to_coords(sensor_num, xyz, dist):
    coords = [0, xyz[1], xyz[2]]
    dim = 1
    if sensor_num in [3, 4, 5]:
        dim = 2
        dist = -dist
    else:
        if sensor_num in [6, 7, 8]:
            dist = -dist
    coords[dim] += dist

    return coords

def read_logs_file(filename):
    data = [[], [], [], [], [], [], [], [], []]

    file = open(filename)
    strings = file.readlines()
    file.close

    i_start = 0
    while strings[i_start] != "do slow run\n":
        i_start += 1

    for j in range(i_start + 1, len(strings)):
        strings[j] = strings[j].replace("\n", "")
        dists = strings[j].split("\t")

        if "STM2 i do slow run and finish" in dists:
            break
        
        for k in range(len(dists)):
            if dists[k] != "":
                dot = dist_to_coords(k, sensors_xyz[k], int(dists[k]))
                dot[0] = (j - i_start - 1) * IMS
                
                data[k].append(dot)
    
    return data

def sensdata_to_coords(data):
    coords = [[], [], []]

    for dot in data:
        coords[0].append(dot[0])
        coords[1].append(dot[1])
        coords[2].append(dot[2])

    return coords

def sensdata_to_2d_coords(data):
    coords = [[], []]

    for dot in data:
        coords[0].append(dot[0])
        coords[1].append(dot[1])

    return coords

## Поиск линий и фильтрация

In [4]:
def calculate_sens_lines(sens_num, data, treshold, fixed_dim=-1):
    lines = {}
    line = [data[0]]
    meds = []
    dim = 1
    if sens_num in [3, 4, 5]:
        dim = 2

    if fixed_dim != -1:
        dim = fixed_dim

    for i in range(1, len(data)):
        goal_axis_difference = data[i][dim] - data[i - 1][dim]
            
        if abs(goal_axis_difference) < treshold:
            line.append(data[i])

        else:
            med = np.mean([x[dim] for x in line])
            meds.append(med)
            lines[med] = line
            line = [data[i]]
    
    med = np.mean([x[dim] for x in line])
    meds.append(med)
    lines[med] = line
    line = [data[i]]
            
    return meds, lines

In [5]:
def get_all_clean(sens_num, data, lines_treshold, clear_num_treshold, filter_min, filter_max):
    all_clean = []
    line = [data[0]]
    dim = 1
    if sens_num in [3, 4, 5]:
        dim = 2

    for i in range(1, len(data)):
        goal_axis_difference = data[i][dim] - data[i - 1][dim]
            
        if abs(goal_axis_difference) < lines_treshold:
            line.append(data[i])

        else:
            med = np.mean([x[dim] for x in line])
            if len(line) >= clear_num_treshold and med <= filter_max and med >= filter_min:
                all_clean += line
            line = [data[i]]
    return all_clean

## Оценка габаритов

In [6]:
def approximate_horisontal_data(clean_data):
    if len(clean_data) > 0:
        x = np.array(clean_data[0])
        y = np.array(clean_data[1])
        A = np.vstack([x, np.ones(len(x))]).T

        m, c = np.linalg.lstsq(A, y, rcond=-1)[0]

        return m, c

    return 0, 0

In [7]:
def dot(vA, vB):
    return vA[0]*vB[0]+vA[1]*vB[1]

def ang(lineA, lineB):
    vA = [(lineA[0][0]-lineA[1][0]), (lineA[0][1]-lineA[1][1])]
    vB = [(lineB[0][0]-lineB[1][0]), (lineB[0][1]-lineB[1][1])]
    dot_prod = dot(vA, vB)
    magA = dot(vA, vA)**0.5
    magB = dot(vB, vB)**0.5
    cos_ = dot_prod/magA/magB
    angle = math.acos(dot_prod/magB/magA)
    ang_deg = math.degrees(angle)%360
    
    if ang_deg-180>=0:
        return 360 - ang_deg
    else: 
        return ang_deg

In [8]:
def clean_line(all_clean, line_coords, sens_num):
    all_clean, line_coords = clean_large_y_outliers(all_clean, sens_num)
    all_clean, line_coords = clean_large_x_outliers(all_clean, sens_num)
    
    return all_clean, line_coords

def clean_large_x_outliers(data, sens_num):
    meds, lines = calculate_sens_lines(sens_num, data, separate_treshold, 0)   
    max_len = 0
    max_med = 0

    for i in range(len(meds)):
        if len(lines[meds[i]]) > max_len:
            max_len = len(lines[meds[i]])
            max_med = i
    line = lines[meds[max_med]]
    left_dot = line[0]
    right_dot = line[-1]
    
    for i in range(max(max_med - 1, 0), -1, -1):
        if left_dot[0] - lines[meds[i]][-1][0] <= connect_treshold or len(lines[meds[i]]) > 50:
            line = lines[meds[i]] + line
            left_dot = line[0]
    for i in range(max_med + 1, len(meds)):
        if lines[meds[i]][0][0] - right_dot[0] <= connect_treshold or len(lines[meds[i]]) > 50:
            line = line + lines[meds[i]]
            right_dot = line[-1]
    
    if len(line) >= 66:
        line_coords = sensdata_to_2d_coords(line)
        return line, line_coords
    else:
        return [], []

def clean_large_y_outliers(data, sens_num):
    meds, lines = calculate_sens_lines(sens_num, data, y_separate_treshold, 1)   
    max_len = 0
    max_med = 0

    for i in range(len(meds)):
        if len(lines[meds[i]]) > max_len:
            max_len = len(lines[meds[i]])
            max_med = i

    line = lines[meds[max_med]]
    left_dot = line[0]
    right_dot = line[-1]
    
    for i in range(max(max_med - 1, 0), -1, -1):
        if abs(lines[meds[i]][-1][1] - left_dot[1]) <= y_connect_treshold or len(lines[meds[i]]) > 50:
            line = lines[meds[i]] + line
            left_dot = line[1]
    for i in range(max_med + 1, len(meds)):
        if abs(lines[meds[i]][0][1] - right_dot[1]) <= y_connect_treshold or len(lines[meds[i]]) > 50:
            line = line + lines[meds[i]]
            right_dot = line[-1]
    line_coords = sensdata_to_2d_coords(line)
    return line, line_coords

def filter_hori_outliers(log_data, sens_num):
    test_data = get_all_clean(sens_num, log_data, 5, 60, 10, 300)

    if len(test_data) > 100:
        all_clean = [[dot[0], dot[1]] for dot in log_data if dot[1] >= 10 and dot[1] <= 300]
        line_coords = sensdata_to_2d_coords(all_clean)

        clf = IsolationForest(max_features=max_features, n_estimators=estimators, random_state=42).fit_predict(all_clean)

        all_clean = [all_clean[i] for i in range(len(all_clean)) if clf[i] == 1]
        line_coords = sensdata_to_2d_coords(all_clean)

        all_clean, line_coords = clean_line(all_clean, line_coords, sens_num)
    
    else:
        return [], []

    return all_clean, line_coords

In [9]:
def filter_len_outputs(total_lengths, lengths, second_lengths, angles, average_y, min_treshold):
    vital_total_lengths = [total_lengths[i] for i in range(0, len(total_lengths)) if total_lengths[i] != 0 and total_lengths[8 - i] != 0]
    vital_lengths = [lengths[i] for i in range(0, len(lengths)) if lengths[i] != 0 and lengths[8 - i] != 0]
    vital_second_lengths = [second_lengths[i] for i in range(0, len(second_lengths)) if second_lengths[i] != 0 and second_lengths[8 - i] != 0]
    total_len_borders = [np.median(vital_total_lengths) + max(min_treshold, np.std(vital_total_lengths)), np.median(vital_total_lengths) - max(min_treshold, np.std(vital_total_lengths))]
    len_borders = [np.median(vital_lengths) + max(min_treshold, np.std(vital_lengths)), np.median(vital_lengths) - max(min_treshold, np.std(vital_lengths))]
    second_len_borders = [np.median(vital_second_lengths) + max(min_treshold, np.std(vital_second_lengths)), np.median(vital_second_lengths) - max(min_treshold, np.std(vital_second_lengths))]

    total_rem_ind = [i for i in range(0, len(total_lengths)) if total_lengths[i] > total_len_borders[0] or total_lengths[i] < total_len_borders[1] and total_lengths[8 - i] == 0]
    first_rem_ind = [i for i in range(0, len(lengths)) if lengths[i] > len_borders[0] or lengths[i] < len_borders[1] and lengths[8 - i] == 0]
    second_rem_ind = [i for i in range(0, len(second_lengths)) if second_lengths[i] > second_len_borders[0] or second_lengths[i] < second_len_borders[1] and second_lengths[8 - i] == 0]

    if np.std(total_lengths) > 0:
        total_lengths = [total_lengths[i] for i in range(0, len(total_lengths)) if i not in total_rem_ind]
        angles = [angles[i] for i in range(0, len(angles)) if i not in total_rem_ind]
        for index in total_rem_ind:
            average_y[index] = 0
    if np.std(lengths) > 0:
        lengths = [lengths[i] for i in range(0, len(lengths)) if i not in first_rem_ind]
    if np.std(second_lengths) > 0:
        second_lengths = [second_lengths[i] for i in range(0, len(second_lengths)) if i not in second_rem_ind]
    total_lengths = [total_lengths[i] for i in range(0, len(total_lengths)) if total_lengths[i] != 0]
    lengths = [lengths[i] for i in range(0, len(lengths)) if lengths[i] != 0]
    second_lengths = [second_lengths[i] for i in range(0, len(second_lengths)) if second_lengths[i] != 0]
    angles = [angles[i] for i in range(0, len(angles)) if angles[i] != 0]

    return total_lengths, lengths, second_lengths, angles, average_y

def approximate_hori_length_angle(log_data):
    total_lengths = [0] * 9
    angles = [0] * 9
    average_y = [0] * 9
    lengths = [0] * 9
    second_lengths = [0] * 9

    for sens_num in [0, 1, 2, 6, 7, 8]:
            if len(log_data[sens_num]) > 0:    
                if sens_num in [0, 1, 2]:
                    max_j = [0, 300]
                else:
                    max_j = [0, 0]

                all_clean, line_coords = filter_hori_outliers(log_data[sens_num], sens_num)

                if len(all_clean) > 0:
                    for i in range(len(line_coords[0])):
                        if sens_num in [0, 1, 2] and line_coords[1][i] < max_j[1]:
                            max_j = [line_coords[0][i], line_coords[1][i]]
                        elif sens_num in [6, 7, 8] and line_coords[1][i] > max_j[1]:
                            max_j = [line_coords[0][i], line_coords[1][i]]
                        
                    coord_len = line_coords[0][-1] - line_coords[0][0]

                    if (max_j == [line_coords[0][0], line_coords[1][0]] or max_j == [line_coords[0][-1], line_coords[1][-1]]) and coord_len > 35:
                        m, c = approximate_horisontal_data(line_coords)
                        total_lengths[sens_num] = line_coords[0][-1] - line_coords[0][0]
                        angles[sens_num] = abs(m)
                        average_y[sens_num] = np.mean(line_coords[1])
                    else:
                        angle = ang([[line_coords[0][0], line_coords[1][0]], max_j], [max_j, [line_coords[0][-1], line_coords[1][-1]]])      
                        dot_len = len(line_coords[0]) * IMS

                        if 110 > angle > 70:
                            angle_1, c1 = approximate_horisontal_data([[line_coords[0][0], max_j[0]], [line_coords[1][0], max_j[1]]])
                            angle_2, c2 = approximate_horisontal_data([[max_j[0], line_coords[0][-1]], [max_j[1], line_coords[1][-1]]])
                            first_len = ((max_j[0] - line_coords[0][0]) ** 2 + (max_j[1] - line_coords[1][0] + 0) ** 2) ** (1/2)
                            second_len = ((max_j[0] - line_coords[0][-1]) ** 2 + (max_j[1] - line_coords[1][-1] + 0) ** 2) ** (1/2)

                            if sens_num in [0, 1, 2]:
                                norm_term = 300
                            else:
                                norm_term = 0
                            coef_1 = abs(max_j[1] - line_coords[1][0]) / abs(max_j[1] - norm_term)
                            coef_2 = abs(max_j[1] - line_coords[1][-1]) / abs(max_j[1] - norm_term)

                            first_appendix = ((Y_Calibration * coef_1 / angle_1) ** 2 + (Y_Calibration * coef_1) ** 2) ** (1/2)
                            second_appendix = ((Y_Calibration * coef_2 / angle_2) ** 2 + (Y_Calibration * coef_2) ** 2) ** (1/2)

                            if first_appendix <= first_len:
                                first_len += first_appendix
                            if second_appendix <= second_len:
                                second_len += second_appendix

                            if first_len > 35 and second_len > 35:
                                if sens_num in [0, 1, 2]:
                                    lengths[sens_num] = first_len
                                    second_lengths[sens_num] = second_len
                                else:
                                    lengths[sens_num] = second_len
                                    second_lengths[sens_num] = first_len
                            elif coord_len < 2 * dot_len and coord_len > 35:
                                total_lengths[sens_num] = coord_len
                                m, c = approximate_horisontal_data(line_coords)
                                angles[sens_num] = abs(m)
                                average_y[sens_num] = np.mean(line_coords[1])

                        elif coord_len < 2 * dot_len and coord_len > 35:
                            total_lengths[sens_num] = coord_len
                            m, c = approximate_horisontal_data(line_coords)
                            angles[sens_num] = abs(m)
                            average_y[sens_num] = np.mean(line_coords[1])

    total_lengths, lengths, second_lengths, angles, average_y = filter_len_outputs(total_lengths, lengths, second_lengths, angles, average_y, 10)
    if len(average_y) > 0:
        average_lower_y = np.mean([average_y[i] for i in range(0, 3) if average_y[i] != 0])
        average_upper_y = np.mean([average_y[i] for i in range(6, 9) if average_y[i] != 0])
        approximated_width = abs(average_upper_y - average_lower_y) + Y_Calibration

    return {"lengths" : lengths, "approximated length" : np.median(lengths), "second lengths" : second_lengths, 
            "approximated second lengths" : np.median(second_lengths), "total lengths" : total_lengths, 
            "approximated total lengths" : np.mean(total_lengths), "angles" : angles, 
            "average_y" : average_y, "approximated width" : approximated_width}

In [10]:
def filter_height_outputs(heights):
    heights_rem_ind = [i for i in range(0, len(heights)) if heights[i] < np.mean(heights)]

    if np.std(heights) > 0:
        heights = [heights[i] for i in range(0, len(heights)) if i not in heights_rem_ind]

    return heights

def approximate_height(log_data):
    heights = []

    for i in [3, 4, 5]:
        all_clean = get_all_clean(i, log_data[i], verticular_treshold, verticular_len, 10, 300)

        if len(all_clean) > 0:
            clean_coords_data = sensdata_to_coords(all_clean)
            heights.append(np.mean(clean_coords_data[2]) + Z_calibration)    

    heights = filter_height_outputs(heights)   

    return {"heights" : heights, "mean height" : np.mean(heights)}

In [11]:
def get_shape(log_data):
    len_res = approximate_hori_length_angle(log_data)
    if len(len_res["lengths"]) >= len(len_res["total lengths"]):
        approximated_length = round(len_res["approximated length"])
        approximated_width =  round(len_res["approximated second lengths"])
    else:
        if len(len_res["total lengths"]) > 0:
            length = len_res["approximated total lengths"]
            angle = np.mean(len_res["angles"])
            approximated_length = (length ** 2 + (length * angle) ** 2) ** (1/2)
            approximated_length = round(approximated_length)
            approximated_width =  round(len_res["approximated width"])
    height_res = approximate_height(log_data) 
    approximated_height = round(height_res["mean height"])
    return approximated_length, approximated_width, approximated_height

In [12]:
def get_all_lengths():
    shapes = []
    time_lens = []
    
    for letter in Filenames.keys():
                for state_num in Filenames[letter].keys():
                    for logs_num in Filenames[letter][state_num]:
                        filename = file_dir + letter + "_" + str(state_num) + "_" + str(logs_num) + ".log"
                        short_name = letter + "_" + str(state_num) + "_" + str(logs_num) + ".log"
                        #print(short_name)
                        log_data = read_logs_file(filename)
                        start = time.time()
                        approximated_length, approximated_width, approximated_height = get_shape(log_data)
                        time_lens.append(time.time() - start)
                        errors = [approximated_length - Trues[letter][state_num][0], approximated_width - Trues[letter][state_num][1], approximated_height - Trues[letter][state_num][2]]
                        ratio = []
                        for i in range(len(errors)):
                            if errors[i] == 0:
                                 ratio.append(0)
                            else:
                                ratio.append(abs(errors[i] / Trues[letter][state_num][i]))
                        shapes.append({"file" : short_name, "true" : Trues[letter][state_num], "len" : approximated_length, "errors" : errors, "ratio" : ratio})
    return shapes, np.mean(time_lens)

In [17]:
def run_tests():
    shapes, avg_time = get_all_lengths()
    analysis = []
    soft_analysis_20 = []
    soft_analysis_30 = []
    soft_analysis_40 = []
    results = []
    ratios = []

    for log in shapes:
        if max(log["errors"]) >= 0 or min(log["errors"]) <= 0:
            analysis.append({"file" : log["file"], "abs" : log["errors"]})
            results.append(log["file"].replace(file_dir, "") + " " + str(log["errors"]))
        if max(log["errors"]) > 10 or min(log["errors"]) < -10:
            soft_analysis_20.append({"file" : log["file"], "abs" : str(log["errors"])})
        if max(log["errors"]) > 15 or min(log["errors"]) < -15:
            soft_analysis_30.append({"file" : log["file"], "abs" : str(log["errors"])})
        if max(log["errors"]) > 20 or min(log["errors"]) < -20:
            soft_analysis_40.append({"file" : log["file"], "abs" : str(log["errors"])})
        ratios += log["ratio"]

    hard_counter = 0
    for letter in Filenames.keys():
                    for state_num in Filenames[letter].keys():
                        for logs_num in Filenames[letter][state_num]:
                                hard_counter += 1
    print("Всего файлов:", hard_counter)
    print("Правильно определено (0 <= отклонение <= 10) Объектов:", len(analysis) - len(soft_analysis_20))

    print("Ошибка  (10 < отклонение <= 15) Объектов:", len(soft_analysis_20) - len(soft_analysis_30))
    
    print("Ошибка  (15 < отклонение <= 20) Объектов:", len(soft_analysis_30) - len(soft_analysis_40))

    print("Ошибка  (20 < отклонение) Объектов:", len(soft_analysis_40))

    max_error = 0
    for item in analysis:
        for error in item["abs"]:
            if abs(error) > max_error:
                 max_error = abs(error)

    print("Максимальный модуль ошибки:", max_error)

    print("Средняя величина отклонения, %:", np.mean(ratios))

    print("Среднее время определения габаритов:", avg_time, "сек.")

    return results

In [18]:
run_tests()

Всего файлов: 104
Правильно определено (0 <= отклонение <= 10) Объектов: 74
Ошибка  (10 < отклонение <= 15) Объектов: 11
Ошибка  (15 < отклонение <= 20) Объектов: 14
Ошибка  (20 < отклонение) Объектов: 5
Максимальный модуль ошибки: 36
Средняя величина отклонения, %: 0.046392315834949245
Среднее время определения габаритов: 0.5278365107683035 сек.


['A_1_1.log [1, 8, 1]',
 'A_1_2.log [-2, 16, -1]',
 'A_1_3.log [-2, 15, 1]',
 'A_2_1.log [2, 3, 5]',
 'A_2_2.log [11, -9, 6]',
 'A_2_3.log [0, 10, 6]',
 'A_3_1.log [-2, 3, 2]',
 'A_3_2.log [-1, 0, 1]',
 'A_3_3.log [2, 1, -1]',
 'B_1_1.log [1, 9, 17]',
 'B_1_2.log [-2, 10, 17]',
 'B_1_3.log [0, 9, 19]',
 'B_2_1.log [-1, 1, 19]',
 'B_2_2.log [2, -2, 19]',
 'B_2_3.log [-20, 7, 19]',
 'B_3_1.log [-2, 2, 6]',
 'B_3_2.log [-1, 5, 6]',
 'B_3_3.log [1, 3, 6]',
 'B_4_1.log [22, -32, 6]',
 'B_4_2.log [4, -15, 6]',
 'B_4_3.log [3, -11, 5]',
 'C_1_1.log [-2, -6, 3]',
 'C_1_2.log [-2, -5, 2]',
 'C_2_1.log [5, -5, 3]',
 'C_2_2.log [7, -14, 2]',
 'C_2_3.log [-2, 6, 3]',
 'C_3_1.log [-6, 0, -3]',
 'C_3_2.log [-10, -1, -4]',
 'C_3_3.log [-4, -5, -5]',
 'C_4_1.log [-2, -5, -4]',
 'C_4_2.log [-17, 3, -5]',
 'C_4_3.log [-10, -3, -5]',
 'D_1_1.log [-3, 5, 3]',
 'D_1_2.log [-4, 4, 2]',
 'D_1_3.log [-5, 4, 3]',
 'D_2_1.log [14, -1, 2]',
 'D_2_2.log [-1, 4, 2]',
 'D_2_3.log [-6, -8, 1]',
 'D_3_1.log [-2, 5, -